In [50]:
import pandas as pd
import numpy as np
import math

In [4]:
articles_df = pd.read_csv('./shared_articles.csv')
articles_df = articles_df.loc[articles_df['eventType'] == 'CONTENT SHARED']

In [6]:
articles_df.head(2)

timestamp       eventType            contentId       authorPersonId  \
1  1459193988  CONTENT SHARED -4110354420726924665  4340306774493623681   
2  1459194146  CONTENT SHARED -7292285110016212249  4340306774493623681   

       authorSessionId authorUserAgent authorRegion authorCountry contentType  \
1  8940341205206233829             NaN          NaN           NaN        HTML   
2  8940341205206233829             NaN          NaN           NaN        HTML   

                                                 url  \
1  http://www.nytimes.com/2016/03/28/business/dea...   
2  http://cointelegraph.com/news/bitcoin-future-w...   

                                               title  \
1  Ethereum, a Virtual Currency, Enables Transact...   
2  Bitcoin Future: When GBPcoin of Branson Wins O...   

                                                text lang  
1  All of this work is still very early. The firs...   en  
2  The alarm clock wakes me at 8:00 with stream o...   en

In [62]:
interactions_df = pd.read_csv('./users_interactions.csv')
interactions_df.head(5)

timestamp eventType            contentId             personId  \
0  1465413032      VIEW -3499919498720038879 -8845298781299428018   
1  1465412560      VIEW  8890720798209849691 -1032019229384696495   
2  1465416190      VIEW   310515487419366995 -1130272294246983140   
3  1465413895    FOLLOW   310515487419366995   344280948527967603   
4  1465412290      VIEW -7820640624231356730  -445337111692715325   

             sessionId                                          userAgent  \
0  1264196770339959068                                                NaN   
1  3621737643587579081  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...   
2  2631864456530402479                                                NaN   
3 -3167637573980064150                                                NaN   
4  5611481178424124714                                                NaN   

  userRegion userCountry  
0        NaN         NaN  
1         NY          US  
2        NaN         NaN  
3        NaN         NaN  
4        NaN         NaN

In [63]:
event_type_strength = {
   'VIEW': 1.0,
   'LIKE': 2.0, 
   'BOOKMARK': 2.5, 
   'FOLLOW': 3.0,
   'COMMENT CREATED': 4.0,  
}

In [64]:
interactions_df['eventStrength'] = interactions_df['eventType'].apply(lambda x: event_type_strength[x])

In [65]:
interactions_df.head()

timestamp eventType            contentId             personId  \
0  1465413032      VIEW -3499919498720038879 -8845298781299428018   
1  1465412560      VIEW  8890720798209849691 -1032019229384696495   
2  1465416190      VIEW   310515487419366995 -1130272294246983140   
3  1465413895    FOLLOW   310515487419366995   344280948527967603   
4  1465412290      VIEW -7820640624231356730  -445337111692715325   

             sessionId                                          userAgent  \
0  1264196770339959068                                                NaN   
1  3621737643587579081  Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_2...   
2  2631864456530402479                                                NaN   
3 -3167637573980064150                                                NaN   
4  5611481178424124714                                                NaN   

  userRegion userCountry  eventStrength  
0        NaN         NaN            1.0  
1         NY          US            1.0  
2        NaN         NaN            1.0  
3        NaN         NaN            3.0  
4        NaN         NaN            1.0

In [66]:
users_interactions_count_df = interactions_df.groupby(['personId','contentId']).size().groupby('personId').size()

In [67]:
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df > 5].reset_index()[['personId']]

In [68]:
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df,
                      how='right',
                      left_on = 'personId',
                      right_on = 'personId')

In [69]:
def smooth_user_preference(x):
    return math.log(1+x, 2)

In [70]:
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['personId', 'contentId'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()

In [72]:
interactions_full_df.head(10)

personId            contentId  eventStrength
0 -9223121837663643404 -8949113594875411859       1.000000
1 -9223121837663643404 -8377626164558006982       1.000000
2 -9223121837663643404 -8208801367848627943       1.000000
3 -9223121837663643404 -8187220755213888616       1.000000
4 -9223121837663643404 -7423191370472335463       3.169925
5 -9223121837663643404 -7331393944609614247       1.000000
6 -9223121837663643404 -6872546942144599345       1.000000
7 -9223121837663643404 -6728844082024523434       1.000000
8 -9223121837663643404 -6590819806697898649       1.000000
9 -9223121837663643404 -6558712014192834002       1.584963